# [*Lab Project Part 2*]() CNNs for Image Classification
------------------------------

### General Guideline
1. Aim:
    - *Understand  the  basic  Image  Classification/Recognition  pipeline  and  the  data-driven  approach (train/predict stages).*
    - *Get used to one of deep learning frameworks (e.g. PyTorch).*
2. Prerequisite:
    - *Familiar with python and relevant packages.*
    - *Known the basic knowledge of Convolutional Neural Networks*

### PyTorch versions
we assume that you are using latest PyTorch version(>=1.4)

### PyTorch Tutorial & Docs
You can learn pytorch from the [tutorial link](https://pytorch.org/tutorials/). The Docs information can be searched at [Docs](https://pytorch.org/docs/stable/index.html). In this assignments, we wish you to form the basic capability of using one of the well-known frameworks for deep learning tasks.  

## Session 1: Image Classifiation on CIFAR 10
### Install pytorch and run the given codes

In [1]:
#####################################################
# referenced codes: https://pytorch.org/tutorials/
# referenced codes: http://cs231n.stanford.edu/
# referenced codes: https://cs.stanford.edu/~acoates/stl10/
######################################################
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

####  *` Q2.1: test dataloader and show the images of each class  of CIFAR10`*  

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# get some random training images 
#at least one of each class
while True:
    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    if len(set(labels)) >= 10: break

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
for i in range (4):
    print(' '.join('%5s' % classes[labels[j]] for j in range(i*8, i*8+8)))

####  *` Q2.2: Architecture understanding. Implement architecture of TwolayerNet and ConvNet.`*  

In [2]:
class TwolayerNet(nn.Module):
    # assign layer objects to class attributes
    # nn.init package contains convenient initialization methods
    # http://pytorch.org/docs/master/nn.html#torch-nn-init
    def __init__(self,input_size ,hidden_size ,num_classes ):
        '''
        :param input_size: 3*32*32
        :param hidden_size: decide by yourself e.g. 1024, 512, 128 ...
        :param num_classes: 
        '''
        super(TwolayerNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self,x):
        # flatten
        x = x.view(x.shape[0], -1)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores


In [ ]:
class ConvNet(nn.Module):
    # Complete the code using LeNet-5
    # reference: https://ieeexplore.ieee.org/document/726791
    def __init__(self):
        
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

        
    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.avg_pool2d(x, 2, 2)
        x = F.tanh(self.conv2(x))
        x = F.avg_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [ ]:
## exmample. You can change and modify it if you like.
## use the above defined trainloader directly and train the models 
def train(net, trainloader,epoch=1):
    ###################### Define Loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    t_ls=[]
    ############################### Training
    for epoch in range(epoch):  # loop over the dataset multiple times 
        loss_ep = 0
        net.train()
        for x, l in trainloader:
            x=x.to(device)
            l=l.to(device)
            net.zero_grad()

            # Step 3. Run our forward pass.
            tag_scores = net(x)
            loss = loss_function(tag_scores, l)

            loss_ep+=loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Training data loss", loss_ep)
        t_ls.append(loss_ep)

    print('Finished Training')
    plt.title('Cross Entropy Loss {}'.format(type(net).__name__))
    plt.xlabel('Epoch')
    if torch.cuda.is_available():
        t_ls = torch.tensor(t_ls, device = 'cpu') 
    plt.plot(t_ls.
             numpy(),label="train")
    plt.legend()
    plt.show()

####  Train Two-layer Net

In [ ]:
model = TwolayerNet(3*32*32, 128, 10)
train(model, trainloader, 10)

#### Train ConvNet - LeNet-5

In [ ]:
model_cnn = ConvNet()
train(model_cnn, trainloader, 10)

In [3]:
def valid(net,testloader):
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            net.eval()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))

In [4]:
def valid_class(net,testloader,classes):
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            net.eval()
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(len(classes)):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

####  Test Two-layer Net

In [ ]:
valid(model,testloader)

In [ ]:
valid_class(model,testloader,classes)

#### Test ConvNet - LeNet-5

In [ ]:
valid(model_cnn,testloader)

In [ ]:
valid_class(model_cnn,testloader,classes)

####  *` Q2.3: Preparation of training. Create Dataloader yourself and define Transform, optimizer.`*  

#### *` Complement  CIFAR10_loader()`*

In [ ]:
###  suggested reference: https://pytorch.org/tutorials/recipes/recipes/custom_dataset_transforms_loader.html?highlight=dataloader
# functions to show an image
import pickle
from torch.utils.data import Dataset, DataLoader

class CIFAR10_loader(Dataset):
    ################################
    # Todo: finish the code
    ################################
    def __init__(self,root,train=True,transform = None):
        if train:
            for i in range(5):
                batch = self.unpickle(root+"data_batch_"+str(i+1))
                temp_data = batch['data']
                temp_labels = batch['labels']
                if i == 0:
                    self.data = temp_data
                    self.labels = temp_labels
                else:
                    self.data = np.concatenate((self.data, temp_data), axis=0)
                    self.labels = np.concatenate((self.labels, temp_labels), axis=0)
        else:
            batch = self.unpickle(root+"test_batch")
            self.data = batch['data']
            self.labels = batch['labels']
        
        self.data = self.reshaped_data(self.data)
        self.transform = transform
        self.length = len(set(self.labels))
    
    def unpickle(self, file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='latin1')
        return dict
    
    def reshaped_data(self, data):
        assert data.shape[1] == 3072
        dim = np.sqrt(1024).astype(int)
        r = data[:, 0:1024].reshape(data.shape[0], dim, dim, 1)
        g = data[:, 1024:2048].reshape(data.shape[0], dim, dim, 1)
        b = data[:, 2048:3072].reshape(data.shape[0], dim, dim, 1)
        reshaped = np.concatenate([r,g,b], -1)
        return reshaped
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        ################################
        # Todo: finish the code
        ################################
        img = self.data[item]
        label = self.labels[item]
        label = torch.tensor(label, dtype=torch.int64)
        #One-hot encoding
        target = np.zeros((self.length))
        target[label - 1] = 1
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.RandomHorizontalFlip(),
     transforms.RandomVerticalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

class Adam:
    def __init__(self, model_params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8):
        self.model_params = list(model_params)
        self.lr = lr
        self.beta_1, self.beta_2 = betas
        self.eps = eps
        self.avg_grads = [torch.zeros_like(p) for p in self.model_params]
        self.avg_sqr_grads = [torch.zeros_like(p) for p in self.model_params]
        self.n_steps = 0
        
    def zero_grad(self):
        for param in self.model_params:
            param.grad = None

    @torch.no_grad()
    def step(self):
        for param, avg_grad, avg_sqr_grad in zip(self.model_params, \
                                                 self.avg_grads, \
                                                 self.avg_sqr_grads):
            
            self.n_steps += 1
            avg_grad.mul_(self.beta_1).add_(param.grad * (1 - self.beta_1))
            avg_sqr_grad.mul_(self.beta_2).add_(param.grad * param.grad * (1 - self.beta_2))
            avg_grad_corrected = avg_grad.div(1 - self.beta_1 ** self.n_steps)
            avg_sqr_grad_corrected = avg_sqr_grad.div(1 - self.beta_2 ** self.n_steps)
            std = avg_sqr_grad_corrected.sqrt().add(self.eps)
            param.sub_(self.lr * avg_grad_corrected / std)

#### *` Train the ConvNet with CIFAR10_loader, transform and optimizer you implemented and compare the results`*

In [ ]:
def train(net, trainloader,epoch=1):
    ###################### Define Loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    if torch.cuda.is_available():
        device = torch.device("cuda")
        optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)
    else:
        device = torch.device("cpu")
        optimizer = Adam(net.parameters())
    net.to(device)
    t_ls=[]
    ############################### Training
    for epoch in range(epoch):  # loop over the dataset multiple times 

        ################################
        # Todo: finish the code
        ################################
        loss_ep = 0
        net.train()
        for x, l in trainloader:
            x=x.to(device)
            l=l.to(device)
            net.zero_grad()

            # Step 3. Run our forward pass.
            tag_scores = net(x)

            loss = loss_function(tag_scores, l)
            
            loss_ep+=loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Training data loss", loss_ep)
        t_ls.append(loss_ep)

    print('Finished Training')
    plt.title('Cross Entropy Loss {}'.format(type(net).__name__))
    plt.xlabel('Epoch')
    if torch.cuda.is_available():
        t_ls = torch.tensor(t_ls, device = 'cpu') 
    plt.plot(t_ls.numpy(),label="train")
    plt.legend()
    plt.show()

In [ ]:
#Call our dataloader
path = "C:/Users/silav/Downloads/labfinal2/data/cifar-10-batches-py/"
transformed_dataset = CIFAR10_loader(path, transform = transform_train)
trainloader = DataLoader(transformed_dataset, batch_size=32, shuffle=True)

# Training Two Layer Net
two_layer_net = TwolayerNet(3*32*32, 128, 10)
print('Training on Cifar10 using Two Layer Net')
train(two_layer_net, trainloader, epoch=10)

# Training ConvNet - LeNet-5
model_cnn = ConvNet()
print('Training on Cifar10 using ConvNet - LeNet-5')
train(model_cnn, trainloader, 10)

#### *` Q2.4 Setting up the hyperparameters.`*

#### *`Play with convNet and TwolayerNet, set up the hyperparameters and reach the accuracy as high as you can`*

### Two Layer Net Hyperparameters pipeline

In [5]:
def train_customization(net, trainloader, loss_function, optimizer, epoch=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    t_ls=[]
    for epoch in range(epoch):  # loop over the dataset multiple times 
        loss_ep = 0
        net.train()
        for x, l in trainloader:
            x=x.to(device)
            l=l.to(device)
            net.zero_grad()

            # Step 3. Run our forward pass.
            tag_scores = net(x)
            loss = loss_function(tag_scores, l)

            loss_ep+=loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("Training data loss", loss_ep)
        t_ls.append(loss_ep)

    print('Finished Training')     

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### Parameters: Batch_size = 64, Loss_function = CrossEntropyLoss, Optimizer = Adam

In [ ]:
model = TwolayerNet(3*32*32, 64, 10)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

In [ ]:
model = ConvNet()
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

#### Parameters: Batch_size = 128, Loss_function = CrossEntropyLoss, Optimizer = Adam

In [ ]:
model = TwolayerNet(3*32*32, 128, 10)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

#### Parameters: Batch_size = 64, Loss_function = MultiMarginLoss, Optimizer = Adam

In [ ]:
model = TwolayerNet(3*32*32, 64, 10)
train_customization(model, trainloader, nn.MultiMarginLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

In [ ]:
model = ConvNet()
train_customization(model, trainloader, nn.MultiMarginLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

#### Parameters: Batch_size = 64, Loss_function = CrossEntropyLoss, Optimizer = Adadelta

In [ ]:
model = TwolayerNet(3*32*32, 64, 10)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adadelta(model.parameters(), lr=1.0, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

In [ ]:
model = ConvNet()train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adadelta(model.parameters(), lr=1.0, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

In [6]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomErasing(),
     transforms.ColorJitter(brightness=0.1, contrast=0.2, saturation=0, hue=0),
     AddGaussianNoise(0.1, 0.08),
     transforms.RandomCrop((120,120)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [ ]:
model = TwolayerNet(3*32*32, 64, 10)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

### Add two layer in each network we have already have

In [ ]:
class FourlayerNet(nn.Module):
    # assign layer objects to class attributes
    # nn.init package contains convenient initialization methods
    # http://pytorch.org/docs/master/nn.html#torch-nn-init
    def __init__(self,input_size ,hidden_size_1, hidden_size_2, hidden_size_3 ,num_classes ):
        '''
        :param input_size: 3*32*32
        :param hidden_size: decide by yourself e.g. 1024, 512, 128 ...
        :param num_classes: 
        '''
        super(FourlayerNet, self).__init__()
        self.input_size = input_size
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.hidden_size_3 = hidden_size_3
        self.num_classes = num_classes
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, hidden_size_3)
        self.fc4 = nn.Linear(hidden_size_3, num_classes) 

        
    def forward(self,x):
        # flatten
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        scores = self.fc4(F.relu(x))
        return scores

In [ ]:
model = FourlayerNet(3*32*32, 256, 128, 64, 10)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

In [ ]:
class ConvNet_plus2(nn.Module):
    # Complete the code using LeNet-5
    # reference: https://ieeexplore.ieee.org/document/726791
    def __init__(self):
        
        super(ConvNet_plus2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.conv4 = nn.Conv2d(32, 64, 5)
        self.fc1   = nn.Linear(256, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

        
    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.avg_pool2d(x, 2, 2)
        x = F.tanh(self.conv3(x))
        x = F.tanh(self.conv4(x))
        x = F.avg_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [ ]:
model_cnn = ConvNet_plus2()
train(model_cnn, trainloader, 10)
valid(model_cnn,testloader)
valid_class(model_cnn,testloader,classes)

#### *` test the accuracy of ConvNet `*

In [ ]:
model_cnn = ConvNet_plus2()
train(model_cnn, trainloader, 10)
valid(model_cnn,testloader)
valid_class(model_cnn,testloader,classes)

#### *`test the accuracy of TwolayerNet`*

In [ ]:
model = FourlayerNet(3*32*32, 256, 128, 64, 100)
train_customization(model, trainloader, nn.CrossEntropyLoss(), optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001), 10)
valid(model,testloader)
valid_class(model,testloader,classes)

## Session 2:  Finetuning the ConvNet
### STL-10 DATASET
> The provided network is trained on a different dataset named CIFAR-10 , which
contains the images of 10 different object categories. The dataset we use throughout the assignment is a subset of STL-10 
with larger sizes and different object classes. So, there is a discrepancy between the
dataset we use to train (CIFAR-10) and test (STL-10) our network. One solution
would be to train the whole network from scratch. However, the number of param-
eters are too large to be trained properly with such few number of images provided.
One solution is to shift the learned weights in a way to perform well on the test
set, while preserving as much information as necessary from the training class.
### In this Session, extract 5 classes from STL training dataset , 
the the label of images can be defined as `{1: 'airplanes',2:'birds',3:'ships',4:'cats',5:'dogs'}`

#### *`Q3.1 create the STL10_Dataset `*

In [ ]:
from stl10_data import *
class STL10_Dataset(Dataset):
    def __init__(self,root,train=True,transform = None):
        ################################
        # Todo: finish the code
        ################################

 

    def __len__(self):
        ################################
        # Todo: finish the code
        ################################

    def __getitem__(self, item):
        ################################
        # Todo: finish the code
        ################################

        return img, target


#### *`Q3.2  Finetuning from ConvNet & train the model and show the results`*

In [ ]:
################################
# Todo: finish the code
################################

###  `Bonus`

In [ ]:
pass